**Step 1: Generate dataset of hand probabilities using Monte Carlo Simulation (10000 simulations generates 440000 hand outcomes).**

**Poker Model using Monte Carlo Simulation**

In [1]:
import numpy as np
import pandas as pd
import itertools
from collections import defaultdict
from enum import Enum

**Define hand functions.**

In [2]:
class Hand(Enum):
  HighCard = 1
  OnePair = 2
  TwoPair = 3
  Trips = 4
  Straight = 5
  Flush = 6
  FullHouse = 7
  Quads = 8
  StraightFlush = 9

In [3]:
# Straight Flush

def checkStraightFlush(hand):
  if checkFlush(hand):
    hand = getFlush(hand)
    if checkStraight(hand) and len(hand) >= 5:
      return True
    else:
      return False
  else:
    return False

# Quads

def checkQuads(hand):
  values = [i[0] for i in hand]
  valueCounts = defaultdict(lambda:0)
  for v in values:
    valueCounts[v] += 1
  if 4 in sorted(valueCounts.values()):
    return True
  return False

def getQuads(hand):
  values = [i[0] for i in hand]
  valueCounts = defaultdict(lambda:0)
  for v in values:
    valueCounts[v] += 1
  return sorted([k for k, v in valueCounts.items() if v == 4], reverse = True)

# Full House

def checkFullHouse(hand):
  values = [i[0] for i in hand]
  valueCounts = defaultdict(lambda:0)
  for v in values:
    valueCounts[v] += 1
  sortedValueCounts = sorted(valueCounts.values())
  if 3 in sortedValueCounts:
    if sortedValueCounts.count(3) > 1:
      return True
    elif 2 in sortedValueCounts:
      return True
  return False

def getFullHouse(hand):
  values = [i[0] for i in hand]
  valueCounts = defaultdict(lambda:0)
  for v in values:
    valueCounts[v] += 1
  sortedValueCounts = sorted(valueCounts.values())
  return sorted([k for k, v in valueCounts.items() if v == 3 or v == 2], reverse = True)

# Flush

def checkFlush(hand):
  suits = [i[1] for i in hand]
  suitCounts = defaultdict(lambda:0)
  for suit in suits:
    suitCounts[suit] += 1
  if sorted(suitCounts.values(), reverse = True)[0] >= 5:
    return True
  else:
    return False

def getFlush(hand):
  suits = [i[1] for i in hand]
  suitCounts = defaultdict(lambda:0)
  for suit in suits:
    suitCounts[suit] += 1
  topSuitCount = sorted(suitCounts.values(), reverse = True)[0]
  topSuit = sorted([k for k, v in suitCounts.items() if v == topSuitCount], reverse = True)[0]
  flushCards = []
  for card in hand:
    if card[1] == topSuit:
      flushCards.append(card)
  return flushCards

# Straight

def fiveConsecutiveCards(numberSet):
  if len(numberSet) < 5:
    return False

  for w, z in itertools.groupby(numberSet, lambda x, y = itertools.count(): next(y) - x):
    grouped = list(z)
    if len(grouped) >= 5:
      return True
  return False

def getHighestConsecutiveCard(numberSet):
  for w, z in itertools.groupby(numberSet, lambda x, y = itertools.count(): next(y) - x):
    grouped = list(z)
    if len(grouped) >= 5:
      return sorted(grouped, reverse = True)[0]

def checkStraight(hand):
  values = [i[0] for i in hand]
  valueCounts = defaultdict(lambda:0)
  for v in values:
    valueCounts[v] += 1

  setOfValues = set(values)
  if fiveConsecutiveCards(setOfValues):
    return True
  else:
    lowStraight = set([14,2,3,4,5])
    if lowStraight.issubset(setOfValues):
      return True
    return False

def getStraightTopCard(hand):
  values = [i[0] for i in hand]
  valueCounts = defaultdict(lambda:0)
  for v in values:
    valueCounts[v] += 1

  setOfValues = set(values)
  if fiveConsecutiveCards(setOfValues):
    return getHighestConsecutiveCard(setOfValues)
  else:
    return 5

# Trips

def checkTrips(hand):
  values = [i[0] for i in hand]
  valueCounts = defaultdict(lambda:0)
  for v in values:
    valueCounts[v] += 1
  if 3 in sorted(valueCounts.values()):
    return True
  else:
    return False

def getTrips(hand):
  values = [i[0] for i in hand]
  valueCounts = defaultdict(lambda:0)
  for v in values:
    valueCounts[v] += 1
  return sorted([k for k, v in valueCounts.items() if v == 3], reverse = True)

# Two-Pair

def checkTwoPair(hand):
  values = [i[0] for i in hand]
  valueCounts = defaultdict(lambda:0)
  for v in values:
    valueCounts[v] += 1
  if sorted(valueCounts.values()).count(2) >= 2:
    return True
  else:
    return False

# One-Pair

def checkOnePair(hand):
  values = [i[0] for i in hand]
  valueCounts = defaultdict(lambda:0)
  for v in values:
    valueCounts[v] += 1
  if 2 in valueCounts.values():
    return True
  else:
    return False

# Pair(s)

def getPairs(hand):
  values = [i[0] for i in hand]
  valueCounts = defaultdict(lambda:0)
  for v in values:
    valueCounts[v] += 1
  return sorted([k for k, v in valueCounts.items() if v == 2], reverse = True)

# High Card

def checkHighCard(hand):
  values = [i[0] for i in hand]
  return sorted(values, reverse = True)

def getHighCards(hand):
  values = [i[0] for i in hand]
  return sorted(values, reverse = True)

In [4]:
# Define hierarchy of rank

def checkHand(hand):
  if checkStraightFlush(hand):
    return 9
  if checkQuads(hand):
    return 8
  if checkFullHouse(hand):
    return 7
  if checkFlush(hand):
    return 6
  if checkStraight(hand):
    return 5
  if checkTrips(hand):
    return 4
  if checkTwoPair(hand):
    return 3
  if checkOnePair(hand):
    return 2
  return 1

# Define hand type by name

def handType(hand):
  if checkStraightFlush(hand):
    return Hand.StraightFlush.name
  if checkQuads(hand):
    return Hand.Quads.name
  if checkFullHouse(hand):
    return Hand.FullHouse.name
  if checkFlush(hand):
    return Hand.Flush.name
  if checkStraight(hand):
    return Hand.Straight.name
  if checkTrips(hand):
    return Hand.Trips.name
  if checkTwoPair(hand):
    return Hand.TwoPair.name
  if checkOnePair(hand):
    return Hand.OnePair.name

  return Hand.HighCard.name

In [5]:
def compareCards(firstHand, secondHand, num = None):
  firstHighCards = getHighCards(firstHand)[:num]
  secondHighCards = getHighCards(secondHand)[:num]

  for i in range(len(firstHighCards)):
    comparison = compare(firstHighCards[i], secondHighCards[i])
    if comparison != 0:
      return comparison
    return 0

def compare(card1, card2):
  if card1 > card2:
    return 1
  elif card1 < card2:
    return 2
  else:
    return 0

def breakTie(firstHand, secondHand):
  handScore = checkHand(firstHand)

  if handScore == 9:
    firstStraight = getStraightTopCard(firstHand)
    secondStraight = getStraightTopCard(secondHand)
    if firstStraight > secondStraight:
      return 1
    else:
      return 2

  if handScore == 8:
    firstQuads = getQuads(firstHand)[0]
    secondQuads = getQuads(secondHand)[0]
    if firstQuads > secondQuads:
      return 1
    else:
      return 2

  if handScore == 7:
    firstHouse = getFullHouse(firstHand)[:2]
    secondHouse = getFullHouse(secondHand)[:2]
    for i in range(len(firstHouse)):
      if firstHouse[i] > secondHouse[i]:
        return 1
      elif secondHouse[i] > firstHouse[i]:
        return 2

  if handScore == 6:
    firstFlush = getFlush(firstHand)
    secondFlush = getFlush(secondHand)
    return compareCards(firstFlush, secondFlush, 5)

  if handScore == 5:
    firstStraight = getStraightTopCard(firstHand)
    secondStraight = getStraightTopCard(secondHand)
    if firstStraight > secondStraight:
      return 1
    elif secondStraight > firstStraight:
      return 2
    else:
      return 0

  if handScore == 4:
    firstTrips = getTrips(firstHand)
    secondTrips = getTrips(secondHand)
    if firstTrips > secondTrips:
      return 1
    elif secondTrips > firstTrips:
      return 2
    else:
      firstHand = list(filter(lambda x: x[0] != firstTrips, firstHand))
      secondHand = (list(filter(lambda x: x[0] != secondTrips, secondHand)))
      return compareCards(firstHand, secondHand, 2)

  if handScore == 3:
    firstPairs = getPairs(firstHand)
    secondPairs = getPairs(secondHand)
    if firstPairs[0] > secondPairs[0]:
      return 1
    elif secondPairs[0] > firstPairs[0]:
      return 2
    elif firstPairs[1] > secondPairs[1]:
      return 1
    elif secondPairs[1] > firstPairs[1]:
      return 2
    else:
      firstHand = list(filter(lambda x: x[0] not in firstPairs, firstHand))
      secondHand = list(filter(lambda x: x[0] not in secondPairs, secondHand))
      return compareCards(firstHand, secondHand, 1)

  if handScore == 2:
    firstPair = getPairs(firstHand)[0]
    secondPair = getPairs(secondHand)[0]
    if firstPair > secondPair:
      return 1
    elif secondPair > firstPair:
      return 2
    else:
      firstHand = list(filter(lambda x: x[0] != firstPair, firstHand))
      secondHand = list(filter(lambda x: x[0] != secondPair, secondHand))
      return compareCards(firstHand, secondHand, 3)

  if handScore == 1:
    return compareCards(firstHand, secondHand)
  return 0

In [6]:
def gameResult(playersHand, otherPlayersHand, board):
  return 'Loss'

def winningResult(playersHands, board):
  bestPlayerHand = board + playersHands[0]
  for hand in playersHands:
    hand = hand + board
    if checkHand(hand) > checkHand(bestPlayerHand):
      bestPlayerHand = hand
    elif checkHand(hand) == checkHand(bestPlayerHand):
      is_tie = breakTie(hand, bestPlayerHand)
      if is_tie == 1:
        bestPlayerHand = hand

  return handType(bestPlayerHand)

**Define suits and values in a deck of cards.**

In [7]:
# Define suit and value

deck = list(itertools.product(range(2, 15), ['H', 'D', 'S', 'C']))

# Building the contents of the game

def holdemSimulation(playersHand, numOtherPlayers, numOfFoldingPlayers = 0):
  playingDeck = deck.copy()
  np.random.shuffle(playingDeck)

  playingDeck = list(filter(lambda x: x != playersHand[0], playingDeck))
  playingDeck = list(filter(lambda x: x != playersHand[1], playingDeck))

  otherPlayersHands = []
  for i in range(numOtherPlayers):
    otherPlayersHands.append([playingDeck.pop(0), playingDeck.pop(0)])

  # Simulating the game (flop, turn, river)
  # Flop is cards 2,3,4, card 1 is burned

  flop = playingDeck[1:4]
  del playingDeck[0:4]

  # Turn is card 2, card 1 is burned

  turn = playingDeck[1:2]
  del playingDeck[0:2]

  # River is card 2, card 1 is burned

  river = playingDeck[1:2]
  del playingDeck[0:2]

  board = flop + turn + river

  # Handle folding

  if numOfFoldingPlayers > 0 and numOfFoldingPlayers < numOtherPlayers:
    foldingPlayers = np.random.randint(1, numOtherPlayers, numOfFoldingPlayers)
    handsToDelete = []
    for num in foldingPlayers:
      handsToDelete.append(otherPlayersHands[num])
    for hand in handsToDelete:
      otherPlayersHands = list(filter(lambda x: x != hand, otherPlayersHands))

  return gameResult(playersHand, otherPlayersHands, board)

In [8]:
secondDeck = list(itertools.product(range(2, 15), ['H', 'D', 'S', 'C']))

def holdemSimulationWinningHand(numOfPlayers):
  secondPlayingDeck = secondDeck.copy()
  np.random.shuffle(secondPlayingDeck)

  playersHands = []
  for i in range(numOfPlayers):
    playersHands.append([secondPlayingDeck.pop(0), secondPlayingDeck.pop(0)])

  flop = secondPlayingDeck[1:4]
  del secondPlayingDeck[0:4]

  turn = secondPlayingDeck[1:2]
  del secondPlayingDeck[0:2]

  river = secondPlayingDeck[1:2]
  del secondPlayingDeck[0:2]

  board = flop + turn + river

  return winningResult(playersHands, board)

In [9]:
def game(playersHand, numOfOtherPlayers, gameSims, numOfFoldingPlayers):
  wins = 0

  for i in range(gameSims):
    result = holdemSimulation(playersHand, numOfOtherPlayers, numOfFoldingPlayers)
    if result == 'Win' or result == 'Tie':
      wins += 1

  winPercentage = (wins / gameSims) * 100
  return winPercentage

In [10]:
def isPocketPair(cards):
  if cards[0][0] == cards[1][0]:
    return True
  else:
    return False

def isSuited(cards):
  if cards[0][1] == cards[1][1]:
    return True
  else:
    return False

def isConnected(cards):
  if (cards[0][0] + 1) == cards[1][0]:
    return True
  elif (cards[0][0] - 1) == cards[1][0]:
    return True
  elif cards[0][0] == 14:
    if cards[1][0] == 2:
      return True
    else:
      return False
  elif cards[0][0] == 2:
    if cards[1][0] == 14:
      return True
    else:
      return False
  else:
    return False

In [11]:
def getSuitedCards(suit, cards):
  potentialCards = list(filter(lambda x: x[1] == suit, cards))
  return potentialCards

def getPairCards(value, cards):
  potentialCards = list(filter(lambda x: x[0] == value, cards))
  return potentialCards

def getConnectedCards(value, cards):
  potentialCards = list(filter(lambda x: x[0] == value or x[0] == value + 1, cards))
  return potentialCards

def generateHand(handType):
  deck = list(itertools.product(range(2, 15), ['H', 'D', 'S', 'C']))
  playingDeck = deck.copy()
  np.random.shuffle(playingDeck)
  playersHand = []

  if handType == 'suited':
    suits = ['H', 'D', 'S', 'C']
    np.random.shuffle(suits)
    suit = suits[0]
    playersHand = getSuitedCards(suit, playingDeck)[:2]
  elif handType == 'pairs':
    values = list(range(2, 15))
    np.random.shuffle(values)
    value = values[0]
    playersHand = getPairCards(value, playingDeck)[2]
  elif handType == 'connected':
    values = list(range(2, 15))
    np.random.shuffle(values)
    value = values[0]
    connectedCards = getConnectedCards(value, playingDeck)
    firstCard = connectedCards[0]
    potentialSecondCards = list(filter(lambda x: x[0] != firstCard[0], connectedCards))
    secondCards = potentialSecondCards[0]
    playersHand = [firstCard, secondCard]
  elif handType == 'connected_suit':
    suits = ['H', 'D', 'S', 'C']
    np.random.shuffle(suits)
    suit = suits[0]
    suitedCards = getSuitedCards(suit, playingDeck)
    values = list(range(2, 15))
    np.random.shuffle(values)
    value = values[0]
    connectedCards = getConnectedCards(value, playingDeck)
    firstCard = connectedCards[0]
    potentialSecondCards = list(filter(lambda x: x[0] != firstCard[0], connectedCards))
    secondCard = potentialSecondCards[0]
    playersHand = [firstCard, secondCard]
  else:
    return 'Unknown hand type!'

  return playersHand

In [12]:
columns = [hand.name for hand in Hand.__members__.values()]
columns.insert(0, 'PlayersCount')
winsDf = pd.DataFrame(columns=columns)
sims = 10000

handsDict = {'StraightFlush': 0, 'Quads': 0, 'FullHouse': 0, 'Flush': 0, 'Straight': 0, 'Trips': 0, 'TwoPair': 0, 'OnePair': 0, 'HighCard': 0}

for n in range(2, 10):
  playersDict = {'PlayersCount': n, 'StraightFlush': 0, 'Quads': 0, 'FullHouse': 0, 'Flush': 0, 'Straight': 0, 'Trips': 0, 'TwoPair': 0,
                 'OnePair': 0, 'HighCard': 0}

  for i in range(sims):
    result = holdemSimulationWinningHand(n)
    playersDict[result] = playersDict[result] + 1

  winsDf = winsDf._append(playersDict, ignore_index=True)

winsDf = winsDf.rename(columns = {'StraightFlush': 'StraightFlush Wins Pocket', 'Quads': 'Quads Wins Pocket', 'FullHouse': 'FullHouse Wins Pocket',
                                  'Flush': 'Flush Wins Pocket', 'Straight': 'Straight Wins Pocket', 'Trips': 'Trips Wins Pocket',
                                  'TwoPair': 'TwoPair Wins Pocket', 'OnePair': 'OnePair Wins Pocket', 'HighCard': 'HighCard Wins Pocket'})

columnsNames = ['StraightFlush Wins Pocket', 'Quads Wins Pocket', 'FullHouse Wins Pocket', 'Flush Wins Pocket', 'Straight Wins Pocket',
                 'Trips Wins Pocket', 'TwoPair Wins Pocket', 'OnePair Wins Pocket', 'HighCard Wins Pocket']

winsDf[columnsNames] = winsDf[columnsNames].apply(lambda x: (x / sims) * 100)

winsDf
winsDf.to_csv('winningPokerHands.csv')

In [13]:
handsDf = pd.DataFrame(columns = ['Pocket Cards', 'Pair', 'Suited', 'Connected', 'Win Pocket 8', 'Win Pocket 7', 'Win Pocket 6',
                                  'Win Pocket 5', 'Win Pocket 4', 'Win Pocket 3', 'Win Pocket 2', 'Win Pocket 1'])

pocketDeck = list(itertools.product(range(2, 15), ['H', 'D', 'S', 'C']))
handCombinations = list(itertools.combinations(pocketDeck, 2))
handCombinations = [list(row) for row in handCombinations]
numOfFoldingPlayers = 0
gameSims = 10000

np.random.shuffle(handCombinations)

for hand in handCombinations:
  handDict = {'Pocket Cards': hand, 'Pair': isPocketPair(hand), 'Suited': isSuited(hand), 'Connected': isConnected(hand)}
  for i in range(1, 9):
    handDict['Win Pocket ' + str(i)] = game(hand, i, gameSims, numOfFoldingPlayers)

  handsDictToDf = pd.DataFrame.from_dict(handDict)
  handsDf = pd.concat([handsDictToDf, handsDf])

handsDf.head()

,Pocket Cards,Pair,Suited,Connected,Win Pocket 1,Win Pocket 2,Win Pocket 3,Win Pocket 4,Win Pocket 5,Win Pocket 6,Win Pocket 7,Win Pocket 8
0,"(2, D)",False,False,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"(11, C)",False,False,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,"(4, C)",False,False,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"(9, H)",False,False,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,"(4, S)",False,True,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
newDeck = list(itertools.product(range(2,15), ['H', 'D', 'S', 'C']))

def holdemPocketCardsSimulation(numOfPlayers):
  playingDeck = newDeck.copy()
  np.random.shuffle(playingDeck)

  playersHands = []
  for i in range(numOfPlayers):
    playerHand = [playingDeck.pop(0), playingDeck.pop(0)]
    playerHand.sort(key = lambda x: x[1])
    playerHand.sort(key = lambda x: x[0])
    playerHand = [str(row[0]) + '' + row[1] for row in playerHand]
    playersHands.append(playerHand)

  playersHands = [str(row[0]) + '/' + row[1] for row in playersHands]
  return playersHands

In [15]:
gameSims = 10000

pocketDeck = list(itertools.product(range(2, 15), ['H', 'D', 'S', 'C']))
pocketDeck = [str(row[0]) + '' + row[1] for row in pocketDeck]
handCombinations = list(itertools.combinations(pocketDeck, 2))
handCombinations = [str(row[0]) + '/' + row[1] for row in handCombinations]

handsDf = pd.DataFrame.from_dict({'Pocket Cards': handCombinations})

for n in range(2, 10):
  gameDict = {key: 0 for key in handCombinations}

  for i in range(gameSims):
    pocketCards = holdemPocketCardsSimulation(n)

    for result in pocketCards:
      reversedResult = result.split(sep = '/')
      resultStr = reversedResult[1] + '/' + reversedResult[0]

      if result in gameDict.keys():
        gameDict[result] += 1
      elif resultStr in gameDict.keys():
        gameDict[resultStr] += 1

  handsDf['Frequency with ' + str(n) + 'players'] = handsDf['Pocket Cards'].map(gameDict)

handsDf.tail(10)

,Pocket Cards,Frequency with 2players,Frequency with 3players,Frequency with 4players,Frequency with 5players,Frequency with 6players,Frequency with 7players,Frequency with 8players,Frequency with 9players
1316,13C/14H,22,22,41,33,41,47,65,76
1317,13C/14D,9,21,33,41,54,45,75,63
1318,13C/14S,10,20,35,35,53,53,59,68
1319,13C/14C,21,23,29,45,59,48,55,78
1320,14H/14D,14,21,28,40,43,51,54,73
1321,14H/14S,13,25,29,37,43,45,64,65
1322,14H/14C,11,30,38,41,41,49,62,62
1323,14D/14S,9,21,32,38,42,48,55,64
1324,14D/14C,13,23,33,31,44,56,47,73
1325,14S/14C,23,25,34,39,41,50,50,82


In [16]:
def splitCards(hand):
  pocket = hand.split('/')
  card1 = pocket[0]
  card1 = (int(card1[:-1]), card1[-1])
  card2 = pocket[1]
  card2 = (int(card2[:-1]), card2[-1])

  return [card1, card2]

handsDf['Pocket Cards Tuple'] = handsDf['Pocket Cards'].apply(splitCards)

handsDf['Pair'] = handsDf['Pocket Cards Tuple'].apply(lambda x: isPocketPair(x))
handsDf['Suited'] = handsDf['Pocket Cards Tuple'].apply(lambda x: isSuited(x))
handsDf['Connected'] = handsDf['Pocket Cards Tuple'].apply(lambda x: isConnected(x))

handsDf = handsDf.drop(columns = ['Pocket Cards Tuple'])

handsDf.tail()

handsDf.to_csv('pocketCardsFrequency.csv')